In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import random
from torchvision import models

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

Device available now: cuda


In [ ]:
image_transforms = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
batchsize = 32
rowsize = 28
colsize = 28
inchannels = 3
kernelsize = 5
stride = 1
padding = 0
poolsize = 3
fc_size1 = 384
fc_size2 = 192
no_classes = 10
epochs = 100
overfit_accuracy = 90

In [ ]:
alex_overfitcounts = []
alex_testaccuracies = []

In [ ]:
train = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=image_transforms)
test = torchvision.datasets.CIFAR10(root='./', train=False, transform=image_transforms)

Files already downloaded and verified


In [ ]:
trainindices = list(range(0, len(train), 50))
testindices = list(range(0, len(test),50))
trainset = torch.utils.data.Subset(train,trainindices)
testset = torch.utils.data.Subset(test,testindices)

In [ ]:
traindata = torch.utils.data.DataLoader(trainset,batch_size=batchsize, shuffle=False, num_workers=2,pin_memory=True)
testdata = torch.utils.data.DataLoader(testset,batch_size=batchsize, shuffle=False, num_workers=2,pin_memory=True)

In [ ]:
train_labels = torch.zeros(len(traindata)*batchsize)
test_labels = torch.zeros(len(testdata)*batchsize)
trainnum = 0
testnum = 0
for i,data in enumerate(traindata):
  for j in range(len(data[1])):
    train_labels[trainnum] = data[1][j]
    trainnum+=1
for i,data in enumerate(testdata):
  for j in range(len(data[1])):
    test_labels[testnum] = data[1][j]
    testnum+=1
  

In [ ]:
transform_inception = transforms.Resize((299,299))
criterion = nn.CrossEntropyLoss().to(device)
# model = SimpleAlex(no_classes).to(device)

In [ ]:
# def training(model,traindata,testdata):
def training(model,traindata,testdata,optimizer,train_labels,test_labels):
  test_accuracies = []
  test_accuracies.append(0)
  test_accuracies.append(0)
  test_accuracies.append(0)
  count=0
  flag=0
  prev = 0
  for ep in range(epochs):
    Loss = 0
    num_correct=0
    tot = 0
    testnum = 0
    trainnum = 0
    for i,data in enumerate(traindata):
      x_data = data[0].to(device)
      labels = data[1].to(device)
      for j in range(len(x_data)):
        labels[j] = train_labels[trainnum]
        trainnum+=1
      optimizer.zero_grad()
      outputs = model(x_data)
      _, pred = torch.max(outputs.data, 1)
      num_correct += (pred == labels).sum().item()
      tot += labels.size(0)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      Loss += loss
    print("Accuracy : ",num_correct/tot)
    print("Loss : ",loss)
    if((num_correct/tot) - prev < 0.001):
      flag=1
    prev =  num_correct/tot
    with torch.no_grad():
      correct = 0
      total = 0
      for i,data in enumerate(testdata):
          images = data[0]
          labels = data[1]
          images = images.to(device)
          labels = labels.to(device)
          for j in range(len(images)):
             labels[j] = test_labels[testnum]
             testnum+=1
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item() 
      test_accuracies.append(100*correct/total)
      if flag==1:
        alex_overfitcounts.append(count+1)
        alex_testaccuracies.append(100*correct/total)
        break
      print("Accuracy of the on test images : ",100 * correct / total)
      count+=1
      if flag==1:
        break

In [ ]:
per = 0
for i in range(6):
  model = models.inception_v3().to(device)
  model.aux_logits = False
  optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum = 0.9)
  train_labels = torch.zeros(len(traindata)*batchsize)
  test_labels = torch.zeros(len(testdata)*batchsize)
  trainnum = 0
  testnum = 0
  for i,data in enumerate(traindata):
    for j in range(len(data[1])):
      train_labels[trainnum] = data[1][j]
      trainnum+=1
  for i,data in enumerate(testdata):
    for j in range(len(data[1])):
      test_labels[testnum] = data[1][j]
      testnum+=1
  print("-------------------------------------")
  localtraining_data = traindata
  localtest_data = testdata
  if per!=0:
    trainls = random.sample(range(0,(len(traindata)*batchsize)),int((len(traindata)*batchsize)*per/100))
    testls = random.sample(range(0,(len(testdata)*batchsize)),int((len(testdata)*batchsize)*per/100))
    trainls.sort()
    testls.sort()
    for i in range(len(trainls)):
      train_labels[trainls[i]] = random.randint(0,9)
    for i in range(len(testls)):
      test_labels[testls[i]] = random.randint(0,9)
  training(model,localtraining_data,localtest_data,optimizer,train_labels,test_labels)
  per+=20

/usr/local/lib/python3.8/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


-------------------------------------
Accuracy :  0.1724
Loss :  tensor(3.0163, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy of the on test images :  22.3
Accuracy :  0.2624
Loss :  tensor(2.2218, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy of the on test images :  28.6
Accuracy :  0.3122
Loss :  tensor(3.1433, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy of the on test images :  32.0
Accuracy :  0.3602
Loss :  tensor(2.4258, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy of the on test images :  34.7
Accuracy :  0.388
Loss :  tensor(2.0160, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy of the on test images :  37.0
Accuracy :  0.4064
Loss :  tensor(2.3318, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy of the on test images :  40.5
Accuracy :  0.463
Loss :  tensor(0.6369, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy of the on test images :  38.5
Accuracy :  0.5058
Loss :  tensor(0.5315, device='cuda:0', grad_fn=<NllLossBackwar

KeyboardInterrupt: ignored

In [ ]:
p = [1,2,3,4,5,6]
plt.plot(p,alex_overfitcounts)
plt.xlabel("Corruption Rate")
plt.ylabel("Overfitting Counts")
plt.title("Inception")

In [ ]:
plt.plot(p,alex_testaccuracies)
plt.xlabel("Corruption Rate")
plt.ylabel("Test Accuracies")
plt.title("Inception")